In [ ]:
# Import libraries and set constants
from dotenv import load_dotenv
import os
import mdai
from mdai.visualize import display_annotations
import cv2
import matplotlib.pyplot as plt
import random
import numpy as np

DEBUG = False

load_dotenv('dot.env')

ACCESS_TOKEN = os.getenv('MDAI_TOKEN')
DATA_DIR = os.getenv('DATA_DIR')
DOMAIN = os.getenv('DOMAIN')
PROJECT_ID = os.getenv('PROJECT_ID')
DATASET_ID = os.getenv('DATASET_ID')
ANNOTATIONS = os.path.join(DATA_DIR, os.getenv('ANNOTATIONS'))
LABEL_ID = os.getenv('LABEL_ID')

print(f"ACCESS_TOKEN={ACCESS_TOKEN}")
print(f"DATA_DIR={DATA_DIR}")
print(f"DOMAIN={DOMAIN}")
print(f"PROJECT_ID={PROJECT_ID}")
print(f"DATASET_ID={DATASET_ID}")
print(f"ANNOTATIONS={ANNOTATIONS}")
print(f"LABEL_ID={LABEL_ID}")

In [ ]:
# Start MD.ai client
mdai_client = mdai.Client(domain=DOMAIN, access_token=ACCESS_TOKEN)

# Download the dataset from MD.ai (or use cached version)
project = mdai_client.project(project_id=PROJECT_ID, path=DATA_DIR)
BASE = project.datasets[0].images_dir

In [ ]:
results = mdai.common_utils.json_to_dataframe(ANNOTATIONS)
annotations_df = results['annotations']

# Filter annotations for the free fluid label
free_fluid_annotations = annotations_df[annotations_df['labelId'] == LABEL_ID]

free_fluid_annotations.head()

# Note on paths for each annotation
# The columns StudyInstanceUID and SeriesInstanceUID are used to form the paths
# {BASE}/{StudyInstanceUID}/{SeriesInstanceUID}.mp4

In [ ]:
# Function to construct the video path
def construct_video_path(base_dir, study_uid, series_uid):
    return os.path.join(base_dir, study_uid, f"{series_uid}.mp4")

# Add video paths to the dataframe using .loc to avoid the SettingWithCopyWarning
free_fluid_annotations = free_fluid_annotations.copy()
free_fluid_annotations.loc[:, 'video_path'] = free_fluid_annotations.apply(
    lambda row: construct_video_path(BASE, row['StudyInstanceUID'], row['SeriesInstanceUID']), axis=1)

# Check if video files exist and add the result to the dataframe using .loc
free_fluid_annotations.loc[:, 'file_exists'] = free_fluid_annotations['video_path'].apply(os.path.exists)

# Count the number of annotations with and without corresponding video files
num_with_files = free_fluid_annotations['file_exists'].sum()
num_without_files = len(free_fluid_annotations) - num_with_files

print(f"Annotations with corresponding video files: {num_with_files}")
print(f"Annotations without corresponding video files: {num_without_files}")


In [ ]:
def display_annotation(row):
    video_path = row['video_path']
    frame_number = int(row['frameNumber'])
    foreground = row['data']['foreground']

    # Load the video file
    cap = cv2.VideoCapture(video_path)
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    ret, frame = cap.read()
    cap.release()

    if ret:
        # Convert the frame to RGB (OpenCV loads in BGR)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Create a copy of the frame for annotation
        frame_annotated = frame_rgb.copy()

        # Draw the annotation on the frame
        for polygon in foreground:
            pts = np.array(polygon, np.int32)
            pts = pts.reshape((-1, 1, 2))
            cv2.polylines(frame_annotated, [pts], isClosed=True, color=(255, 0, 0), thickness=2)
            cv2.fillPoly(frame_annotated, [pts], color=(255, 0, 0, 50))

        # Display the frames
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.imshow(frame_rgb)
        plt.title('Original Frame')

        plt.subplot(1, 2, 2)
        plt.imshow(frame_annotated)
        plt.title('Annotated Frame')

        plt.show()
    else:
        print("Failed to read the frame from the video.")



In [ ]:

# Select five random annotations with corresponding video files
# random_annotations = free_fluid_annotations[free_fluid_annotations['file_exists']].sample(n=5, random_state=42)
random_annotations = free_fluid_annotations[free_fluid_annotations['file_exists']].sample(n=5)

# Display the selected annotations
for index, row in random_annotations.iterrows():
    display_annotation(row)